In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Don't allow any numerical fault. More info: https://numpy.org/doc/stable/user/misc.html
np.seterr(all="raise")

mpl.rcParams["figure.figsize"] = (40, 5)

from numpy import datetime64
from numpy.typing import NDArray
from pandas import Timestamp, Timedelta
from pvlib import location
from pytz import timezone

from lib.boat_model import Boat, Other
from lib.panel_model import Panel
from lib.mppt_model import MPPT
from lib.battery_model import Battery
from lib.esc_model import ESC
from lib.motor_model import Motor
from lib.transmission_model import Transmission
from lib.hull_model import Hull
from lib.energy_controller_model import EnergyController, ConstantPowerController
from lib.boat_data import (
    BoatInputData,
    BoatInputDataSet,
    BoatOutputData,
)
from lib.event_model import Event, ControlledBoat
from lib.event_data import EventInputData, EventResultData, RaceStatus, FixedLapsGoal
from lib.competition_model import Competition, CompetitionResult


In [3]:
# Create the competition
site = location.Location(
    latitude=-26.243602,
    longitude=-48.6417668,
    tz="America/Sao_Paulo",
    altitude=0,
    name="São Francisco do Sul",
)

tzinfo = timezone(site.tz)
competition = Competition(
    name="DSB 2020 - São Francisco do Sul",
    events=[
        Event(
            EventInputData(
                name="Short Race 1",
                description="Prova Curta",
                goal=FixedLapsGoal(
                    total_laps=1,
                    lap_distance=np.float64(786.73),
                    event_duration=Timedelta(hours=1),
                ),
                start=Timestamp(
                    year=2020,
                    month=1,
                    day=29,
                    hour=13,
                    minute=51,
                    second=59,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=1,
                    day=29,
                    hour=15,
                    minute=5,
                    second=32,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
        Event(
            EventInputData(
                name="Long Race 1",
                description="Prova longa",
                goal=FixedLapsGoal(
                    total_laps=3 * 2,
                    lap_distance=np.float64(7081.987),
                    event_duration=Timedelta(hours=5),
                ),
                start=Timestamp(
                    year=2020,
                    month=1,
                    day=30,
                    hour=11,
                    minute=16,
                    second=45,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=1,
                    day=30,
                    hour=14,
                    minute=55,
                    second=30,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
        Event(
            EventInputData(
                name="Relay Race",
                description="Prova de revezamento de pilotos",
                goal=FixedLapsGoal(
                    total_laps=2,
                    lap_distance=np.float64(812.19),
                    event_duration=Timedelta(hours=3),
                ),
                start=Timestamp(
                    year=2020,
                    month=1,
                    day=31,
                    hour=11,
                    minute=23,
                    second=23,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=1,
                    day=31,
                    hour=13,
                    minute=27,
                    second=23,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
        Event(
            EventInputData(
                name="Short Race 2",
                description="Prova curta",
                goal=FixedLapsGoal(
                    total_laps=1,
                    lap_distance=np.float64(4678.39),
                    event_duration=Timedelta(hours=2),
                ),
                start=Timestamp(
                    year=2020,
                    month=2,
                    day=1,
                    hour=10,
                    minute=15,
                    second=15,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=2,
                    day=1,
                    hour=11,
                    minute=37,
                    second=28,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
        Event(
            EventInputData(
                name="Short Race 3",
                description="Prova curta",
                goal=FixedLapsGoal(
                    total_laps=1,
                    lap_distance=np.float64(4678.39),
                    event_duration=Timedelta(hours=1),
                ),
                start=Timestamp(
                    year=2020,
                    month=2,
                    day=1,
                    hour=13,
                    minute=15,
                    second=9,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=2,
                    day=1,
                    hour=13,
                    minute=37,
                    second=28,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
        Event(
            EventInputData(
                name="Sprint Race",
                description="Prova de arrancada",
                goal=FixedLapsGoal(
                    total_laps=1,
                    lap_distance=np.float64(1057.03),
                    event_duration=Timedelta(hours=15),
                ),
                start=Timestamp(
                    year=2020,
                    month=2,
                    day=2,
                    hour=11,
                    minute=58,
                    second=27,
                    tzinfo=tzinfo,
                ).tz_convert(None),
                end=Timestamp(
                    year=2020,
                    month=2,
                    day=2,
                    hour=11,
                    minute=59,
                    second=9,
                    tzinfo=tzinfo,
                ).tz_convert(None),
            )
        ),
    ],
)


In [4]:
event = competition.events[4]
start = event.data.start
end = event.data.end
name = event.data.name

df_pred = pd.read_csv("models/2020/boat_data_1s_pred.csv")
df_pred["timestamp"] = pd.DatetimeIndex(df_pred["timestamp"]).tz_convert(None)
df_pred = df_pred.set_index("timestamp")
df_pred = df_pred[(df_pred.index >= start) & (df_pred.index <= end)]

df_pred

,Unnamed: 0,boat_on,motor_on,dms_on,motor_rpm,motor_dt,bat_v,bat1_v,bat2_v,bat3_v,...,mcb2_io,mcb_po,mcb1_po,mcb2_po,solcast_poa,solcast_AirTemp,solcast_energy,pred_speed,prop_pi,pred_speed2
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-02-01 16:15:09,263896,True,True,False,0.0,0.042023,44.945312,15.726562,15.179688,14.039062,...,0.0,0.0,0.0,0.0,1009.777857,28.0,20365.559098,0.0,0.0,0.0
2020-02-01 16:15:10,263897,True,True,False,6.0,0.058838,45.062500,15.812500,15.179688,14.070312,...,0.0,0.0,0.0,0.0,1009.777857,28.0,20365.559098,0.0,0.0,0.0
2020-02-01 16:15:11,263898,True,True,False,120.0,0.058838,45.039062,15.796875,15.179688,14.062500,...,0.0,0.0,0.0,0.0,1009.777857,28.0,20365.559098,0.0,0.0,0.0
2020-02-01 16:15:12,263899,True,True,False,120.0,0.058838,45.015625,15.796875,15.179688,14.039062,...,0.0,0.0,0.0,0.0,1009.777857,28.0,20365.559098,0.0,0.0,0.0
2020-02-01 16:15:13,263900,True,True,False,120.0,0.062347,45.039062,15.851562,15.179688,14.007812,...,0.0,0.0,0.0,0.0,1009.777857,28.0,20365.559098,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-01 16:37:24,265231,True,True,False,0.0,0.027451,37.304688,13.031250,12.687500,11.585938,...,0.0,0.0,0.0,0.0,713.251384,27.9,20668.356179,0.0,0.0,0.0
2020-02-01 16:37:25,265232,True,True,False,0.0,0.027451,37.328125,13.031250,12.687500,11.609375,...,0.0,0.0,0.0,0.0,713.251384,27.9,20668.356179,0.0,0.0,0.0
2020-02-01 16:37:26,265233,True,True,False,0.0,0.027451,37.289062,13.031250,12.687500,11.570312,...,0.0,0.0,0.0,0.0,713.251384,27.9,20668.356179,0.0,0.0,0.0
